# Shooting the bullets

In this section we will code to allow our ship to shoot bullets after taking input from users. 

We’ll write code that fires a bullet
(a small rectangle) when the player presses the spacebar. Bullets will then travel straight up the screen until they disappear off the top of the screen.


First of all we will add some settings describing the bullets in *settings.py*.

In [ ]:
class Settings():
    """A class to store all settings for Alien Invasion."""
    def __init__(self):
        """Initialize the game's settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800

        # Bullet settings
        self.bullet_width = 3
        self.bullet_height = 15
        self.bullet_color = (230,230,230)


These settings create light grey bullets with a width of 3 pixels and a height of 15 pixels.

The next thing to do is going to be to make a class `Bullet` in a module *bullet.py* to manage the bullets. 

Another concept that you are going to learn about here is **sprites**. A sprite is simply a  graphic that is designed to be part of a larger scene. It can either be a static image or an animated graphic. The reason for using sprites here is when you use **sprites**, you can group related elements in your game and act on all the grouped elements at once. Although in old video games **sprites** used to be the standard way of integrating graphics in video games.

So make a file *bullet.py* with the following code.

In [ ]:
import pygame
from pygame.sprite import Sprite

class Bullet(Sprite):
  """A class to manage bullets fired from the ship"""
  def __init__(self, ai_settings, screen, ship):
    """Create a bullet object at the ship's current position."""
    super().__init__() # connect Sprite and Bullet (Python 3 syntax is used)
    self.screen = screen

    # Create a bullet rect at (0, 0) and then set correct position.
    self.rect = pygame.Rect(0, 0, ai_settings.bullet_width, ai_settings.bullet_height) # create rect at (0,0) with dimensions from ai_settings
    # set the bullets's position according to the ship's position
    self.rect.centerx = ship.rect.centerx
    self.rect.top = ship.rect.top

    # store the colour of the bullet
    self.color = ai_settings.bullet_color


The `Bullet` class inherits from `Sprite`, which we import from the
`pygame.sprite` module. You guys don't need to know much about inheritane. All you need to understantd is, when one class inherits from another, it automatically takes on all the attributes and methods of the first class. So here the `bullet` class can also use the methods and attributes defined in the `Sprite` class. Another thing to note here is, the `super()` function which is a special function that helps Python make connections between the 2 classes. It will create a temporary object of the `Sprite` class through which we can access ts methods and attribues. If you want to learn more about inheritance you can watch [this video](https://www.youtube.com/watch?v=H2SQrZK2nvM&list=PLzMcBGfZo4-l1MqB1zoYfqzlj_HH-ZzXt&index=5)

To create a bullet instance, \_\_init__() needs the **ai_settings**, **screen**, and **ship** instancesand we call super() to inherit properly from `Sprite`.  We then create the bullet’s rect attribute. The bullet is not based on an
image so we have to build a rect from scratch using the `pygame.Rect()` class. This class requires the x- and y-coordinates of the top-left corner of the *rect*, and the width and height of the *rect*. We initialize the rect at (0, 0), but we’ll move it to the correct location in the next two lines, because the bullet’s position is dependent on the ship’s position. We get the width and
height of the bullet from the values stored in ai_settings.

We set the bullet’s **centerx** to be the same as the ship’s **rect.centerx**. The bullet should emerge from the top of the ship, so we set the top of the bullet’s rect to match the top of the ship’s rect, making it look like the bullet is fired from the ship.




Now we add 2 more functions in the Bullet class, one to update the position of bullet i.e. make it move upward(We make it move 3 pixels at a time, a bullet should be faster than the ship) and another to draw the bullet on the screen

In [ ]:
  def update(self):
    """Move the bullet up the screen."""
    self.rect.y -= 3 #Remeber how the coordinates here work

  def draw_bullet(self):
    """Draw the bullet to the screen."""
    pygame.draw.rect(self.screen, self.color, self.rect)


## Storing bullets in a group

Now that we have a Bullet class and the necessary settings defined, we can write code to fire a bullet each time the player presses the spacebar. First, we’ll create a group in *alien_invasion.py* to store all the live bullets so we can manage the bullets that have already been fired. This group will be an instance of the class `pygame.sprite.Group`, which behaves like a list with some extra functionality that’s helpful when building games.

Here's how *alien_invasion.py* will be modified.

In [ ]:
import pygame
from pygame.sprite import Group
--snip--

def run_game():
  --snip--

  # Make a ship.
  ship = Ship(ai_settings, screen)

  # Make a group to store bullets in.
  bullets = Group()

  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update() 
    bullets.update() #update position of all bullets
    gf.update_screen(bg_img, screen, ship, bullets)

run_game()


We import Group from pygame.sprite. We make an instance of Group
and call it bullets. This group is created outside of the while loop so we don’t create a new group of bullets each time the loop cycles.

We pass bullets to `check_events()` and `update_screen()`. We’ll need to work with bullets in `check_events()` when the spacebar is pressed, and we’ll need to update the bullets that are being drawn to the screen in `update_screen()`.

When you call `update()` on a group, the group automatically calls
`update()` for each sprite in the group. 



### Firing the bullets

In game_functions.py, we need to modify `check_keydown_events()` to fire a bullet when the spacebar is pressed. We don’t need to change `check_keyup_events()` because nothing happens when the key is released. We also need to modify `update_screen()` to make sure each bullet is redrawn to the screen before we call `flip()`. Here are the relevant changes to *game_functions.py*:


In [ ]:
import sys
import pygame
from bullet import Bullet

def check_events(ai_settings, screen, ship, bullets):
  """Respond to keypresses and mouse events."""
  --snip--
    elif event.type == pygame.KEYDOWN:
      check_keydown_events(event, ai_settings, screen, ship, bullets) 

    --snip--

def check_keydown_events(event, ai_settings, screen, ship, bullets):
  """respond to keypresses"""
  --snip--
  elif event.key == pygame.K_SPACE:
    # Create a new bullet and add it to the bullets group.
    new_bullet = Bullet(ai_settings, screen, ship)
    bullets.add(new_bullet)


def check_keyup_events(event, ship):
  --snip--

def update_screen(bg_img, screen, ship, bullets):
  """Update images on the screen and flip to the new screen."""
  # Redraw the screen during each pass through the loop.
  screen.blit(bg_img,(0,0))

  # Redraw all bullets behind the ship.
  for bullet in bullets.sprites():
    bullet.draw_bullet()

  # redraw the ship
  ship.blitme()

  # Make the most recently drawn screen visible.
  pygame.display.flip()



The group **bullets** is passed to `check_keydown_events()`. When the
player presses the spacebar, we create a new bullet (a Bullet instance that we name **new_bullet**) and add it to the group **bullets** using the `add()` method.

We need to add **bullets** as well as the other objects needed as parameters in the definition of `check_events()` and `check_keydown_events()`.

We also give the **bullets** parameter to `update_screen()`, which draws the bullets to the screen. The `bullets.sprites()` method returns a list of all **sprites** in the group **bullets**. To draw all fired bullets to the screen, we loop through the **sprites** in bullets and call `draw_bullet()` on each one 

### Deleting old bullets
At the moment, the bullets disappear when they reach the top, but only because Pygame can’t draw them above the top of the screen. The bullets actually continue to exist; their y-coordinate values just grow increasingly negative. This is a problem, because they continue to consume memory and processing power.

To get rid of these bullets, we need to detect when the
bottom value of a bullet’s rect has a value of 0, which indicates the bullet has passed off the top of the screen so we modify alien_invasion.py in the following manner.



In [ ]:
import pygame
from pygame.sprite import Group
--snip--

def run_game():
  --snip--

  # Make a ship.
  ship = Ship(ai_settings, screen)

  # Make a group to store bullets in.
  bullets = Group()

  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update() 
    bullets.update() #update position of all bullets
    
    # Get rid of bullets that have disappeared.
    for bullet in bullets.copy():
      if bullet.rect.bottom <= 0:
        bullets.remove(bullet)

    gf.update_screen(ai_settings, screen, ship, bullets)

run_game()

You shouldn’t remove items from a list or group within a for loop, so
we have to loop over a copy of the group. We use the `copy()` method to set up the **for** loop, which enables us to modify bullets inside the loop. We check each bullet to see whether it has disappeared off the top of the screen. If it has, we remove it from **bullets**. 

After the **for** loop you can add `print(len(bullets))` to see how many bullets are there at an instant and see whether this works as we desired.

### Limiting the Number of Bullets
Many shooting games limit the number of bullets a player can have on the screen at one time to encourage players to shoot accurately. We’ll do the same in Alien Invasion. First, store the number of bullets allowed in settings.py:



In [ ]:
    # Bullet settings
    self.bullet_width = 3
    self.bullet_height = 15
    self.bullet_color = 60, 60, 60
    self.bullets_allowed = 3


This limits the player to three bullets at a time. We’ll use this setting in game_functions.py to check how many bullets exist before creating a new bullet in check_keydown_events():


In [ ]:
def check_keydown_events(event, ai_settings, screen, ship, bullets):
  --snip--
  elif event.key == pygame.K_SPACE:
    # Create a new bullet and add it to the bullets group.
    if len(bullets) < ai_settings.bullets_allowed:
      new_bullet = Bullet(ai_settings, screen, ship)
      bullets.add(new_bullet)


### Refactoring
Now we have written all the code for creating and managing bullets we should refactor it to keep our code as simple as possible. 
#### Create *update_bullets()* function
The bullet management code that we have written in *alien_invasion.py*, should be moved to *game_functions.py*. We add the following function in *game_function.py*:

In [ ]:
def update_bullets(bullets):
  """Update position of bullets and get rid of old bullets."""
  # Update bullet positions.
  bullets.update()
  # Get rid of bullets that have disappeared.
  for bullet in bullets.copy():
    if bullet.rect.bottom <= 0:
      bullets.remove(bullet)


Now the **while** loop in *alien_invasion.py* looks like:

In [ ]:
    # Start the main loop for the game.
    while True:
        gf.check_events(ai_settings, screen, ship, bullets)
        ship.update() 
        gf.update_bullets(bullets)
        gf.update_screen(bg_img, screen, ship, bullets)


#### Create the *fire_bullet()* function
Let’s move the code for firing a bullet to a separate function so we
can use a single line of code to fire a bullet and keep the *elif* block in *check_keydown_events()* simple:


In [ ]:
def check_keydown_events(event, ai_settings, screen, ship, bullets):
  """Respond to keypresses."""
  --snip--
  elif event.key == pygame.K_SPACE:
    fire_bullet(ai_settings, screen, ship, bullets)


def fire_bullet(ai_settings, screen, ship, bullets):
  """Fire a bullet if limit not reached yet."""
  # Create a new bullet and add it to the bullets group.
  if len(bullets) < ai_settings.bullets_allowed:
      new_bullet = Bullet(ai_settings, screen, ship)
      bullets.add(new_bullet)
